In [3]:
import os
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv1D, Conv2D,BatchNormalization, MaxPooling1D,MaxPooling2D,LayerNormalization, Dropout, Flatten, Dense, concatenate ,AveragePooling1D
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.metrics import binary_accuracy
from sklearn.metrics import confusion_matrix, recall_score, roc_curve, roc_auc_score, auc,average_precision_score,matthews_corrcoef
import math
import tensorflow as tf
from tensorflow.keras import losses
from sklearn.utils import compute_class_weight
import random
import itertools
import pickle
np.random.seed(seed=21)
import math

2024-03-20 07:40:23.770199: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-20 07:40:23.814003: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 07:40:23.814045: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 07:40:23.814075: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-20 07:40:23.822903: I tensorflow/core/platform/cpu_feature_g

In [4]:

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

TF_ENABLE_ONEDNN_OPTS=0

In [5]:
#protein_feat_r.pkl,protein_feat_r_pb.pkl,protein_feat_r_pbBFD.pkl,protein_feat_r_TX.pkl,protein_feat_r_XLNet.pkl
win_size=5
emb=''

In [6]:
pathdata='./dataset/'

with open(pathdata+'Train_335.pkl', 'rb') as file:
    train = pickle.load(file) 
with open(pathdata+'Test_60.pkl', 'rb') as file:
    test = pickle.load(file)
with open(pathdata+'Test_20_new.pkl', 'rb') as file:
    test_20 = pickle.load(file)
with open(pathdata+'UBtest_31.pkl', 'rb') as file:
    test_UB_31 = pickle.load(file)
with open(pathdata+'Btest_31.pkl', 'rb') as file:
    test_B_31 = pickle.load(file)


all_data=dict()
for index in train.keys():
    all_data[index]=train[index]
for index in test.keys():
    all_data[index]=test[index]
for index in test_20.keys():
    all_data[index]=test_20[index]
for index in test_UB_31.keys():
    all_data[index]=test_UB_31[index]
for index in test_B_31.keys():
    all_data[index]=test_B_31[index]

seq_all=dict()
for index in all_data.keys():
    temp=all_data[index]
    seq_all[index]=temp[0]
label_all=dict()
for index in all_data.keys():
    temp=all_data[index]
    label_all[index]=temp[1]




In [7]:
out_folder=('./feature_save')


with open(os.path.join(out_folder,f'protein_feat_r{emb}.pkl'), 'rb') as file:
    protein_feat_r = pickle.load(file)
    print('loaded protein_feat_r')
    
with open(os.path.join(out_folder,'protein_feat_one.pkl'), 'rb') as file:
    protein_feat_one = pickle.load(file)
    print('loaded protein_feat_one')


loaded protein_feat_r
loaded protein_feat_one


In [8]:
pathdata='./dataset/'

# pathdata='./dataset/'
with open(pathdata+'Test_70.pkl', 'rb') as file:
    test_70 = pickle.load(file) 




new_dataset=dict()
for index in train.keys():
    new_dataset[index]=train[index]


for index in train.keys():    
    for index1 in test_70.keys():        
        if index == index1:
            del new_dataset[index1]

train=new_dataset

all_data_1=dict()
for index in test_70.keys():
    all_data_1[index]=test_70[index]


seq_all_1=dict()
for index in all_data_1.keys():
    temp=all_data_1[index]
    seq_all_1[index]=temp[0]
label_all_1=dict()
for index in all_data_1.keys():
    temp=all_data_1[index]
    label_all_1[index]=temp[1]



out_folder=('./feature_save')
with open(os.path.join(out_folder,f'protein_feat_pisite_r{emb}.pkl'), 'rb') as file:
    protein_feat_pisite_r = pickle.load(file)
    # print('loaded protein_feat_r')
    
with open(os.path.join(out_folder,'protein_feat_pisite_one.pkl'), 'rb') as file:
    protein_feat_pisite_one = pickle.load(file)
    # print('loaded protein_feat_one')



test_70_index=dict()


for index in test_70.keys():
    test_70_index[index]=0

prot_test_70=dict()
prot_test_70_one=dict()
label_test_70=dict()


for index in test_70_index.keys():
    prot_test_70[index]=protein_feat_pisite_r[index]
    prot_test_70_one[index]=protein_feat_pisite_one[index]
    label_test_70[index]=label_all_1[index]



prot_test_70=np.array(list(prot_test_70.items()),dtype=object)[:,1]
prot_test_70_one=np.array(list(prot_test_70_one.items()),dtype=object)[:,1]
label_test_70=np.array(list(label_test_70.items()),dtype=object)[:,1]


In [9]:
#### windowing for features####

In [10]:
def windowing(features,w_size):   
    to=0
    for index in features:
        b=np.shape(index)[0]
        to=to+b
    
        
    
    a=features[0]
    fea_len=np.shape(a)[1]
    finalout1=np.zeros([to,w_size,fea_len],'float')
    
    l=0
    for i in range(0,len(features)):
        temp_features=features[i]
        
        for j in range( 0, np.shape(temp_features)[0]):
            
            
            for k in range(0,w_size):
                
                k1=int(j+k-((w_size-1)/2))
                
                if k1<0 or k1 > np.shape(temp_features)[0]-1:
                    pass
                else:
                    finalout1[l,k,:]=temp_features[k1,:]
            l=l+1
    finalout1=finalout1.reshape((finalout1.shape[0], finalout1.shape[1],finalout1.shape[2], 1))
    return finalout1



In [11]:
def label( labels):   
    final_label=labels
    
    to=0
    for i in range(0,len(final_label)):
        temp_label=final_label[i]
        for j in range(0,len(temp_label)):
            to=to+1
    
    
    finallabel=np.zeros([to],'int')
    
    l=0
    for i in range(0,len(final_label)):
        # print(i)

        temp_label=final_label[i]
        
        for j in range( 0, len(temp_label)):
            

            finallabel[l]=temp_label[j]
            l=l+1
            
    return  finallabel 

In [12]:
train_index=dict()
test_index=dict()
test_20_index=dict()
test_UB_31_index=dict()
test_B_31_index=dict()

for index in train.keys():
    train_index[index]=0
for index in test.keys():
    test_index[index]=0
for index in test_20.keys():
    test_20_index[index]=0  
for index in test_UB_31.keys():
    test_UB_31_index[index]=0
for index in test_B_31.keys():
    test_B_31_index[index]=0  

In [13]:
prot_test_60=dict()
label_test_60=dict()


for index in test_index.keys():
    prot_test_60[index]=protein_feat_r[index]
    label_test_60[index]=label_all[index]


prot_train_335=dict()
label_train_335=dict()

for index in train_index.keys():
    prot_train_335[index]=protein_feat_r[index]
    label_train_335[index]=label_all[index]

prot_test_20=dict()
label_test_20=dict()

for index in test_20_index.keys():
    prot_test_20[index]=protein_feat_r[index]
    label_test_20[index]=label_all[index]

prot_test_UB_31=dict()
label_test_UB_31=dict()

for index in test_UB_31_index.keys():
    prot_test_UB_31[index]=protein_feat_r[index]
    label_test_UB_31[index]=label_all[index]

prot_test_B_31=dict()
label_test_B_31=dict()

for index in test_B_31_index.keys():
    prot_test_B_31[index]=protein_feat_r[index]
    label_test_B_31[index]=label_all[index]





prot_train_335=np.array(list(prot_train_335.items()),dtype=object)[:,1]
label_train_335=np.array(list(label_train_335.items()),dtype=object)[:,1]


prot_test_60=np.array(list(prot_test_60.items()),dtype=object)[:,1]
label_test_60=np.array(list(label_test_60.items()),dtype=object)[:,1]

prot_test_20=np.array(list(prot_test_20.items()),dtype=object)[:,1]
label_test_20=np.array(list(label_test_20.items()),dtype=object)[:,1] 

prot_test_UB_31=np.array(list(prot_test_UB_31.items()),dtype=object)[:,1]
label_test_UB_31=np.array(list(label_test_UB_31.items()),dtype=object)[:,1] 

prot_test_B_31=np.array(list(prot_test_B_31.items()),dtype=object)[:,1]
label_test_B_31=np.array(list(label_test_B_31.items()),dtype=object)[:,1] 




In [14]:
prot_test_60_one=dict()

for index in test_index.keys():
    prot_test_60_one[index]=protein_feat_one[index]

prot_train_335_one=dict()

for index in train_index.keys():
    prot_train_335_one[index]=protein_feat_one[index]

prot_test_20_one=dict()

for index in test_20_index.keys():
    prot_test_20_one[index]=protein_feat_one[index]


prot_test_UB_31_one=dict()

for index in test_UB_31_index.keys():
    prot_test_UB_31_one[index]=protein_feat_one[index]

prot_test_B_31_one=dict()

for index in test_B_31_index.keys():
    prot_test_B_31_one[index]=protein_feat_one[index]



prot_train_335_one=np.array(list(prot_train_335_one.items()),dtype=object)[:,1]

prot_test_60_one=np.array(list(prot_test_60_one.items()),dtype=object)[:,1]

prot_test_20_one=np.array(list(prot_test_20_one.items()),dtype=object)[:,1]

prot_test_UB_31_one=np.array(list(prot_test_UB_31_one.items()),dtype=object)[:,1]
    
prot_test_B_31_one=np.array(list(prot_test_B_31_one.items()),dtype=object)[:,1]




In [15]:
prot_test_60=windowing(prot_test_60,win_size)# transformer features
prot_test_60_one=windowing(prot_test_60_one,win_size)#one hot  encoding features
label_test_60=label(label_test_60) # lables


prot_train_335  =windowing(prot_train_335,win_size)
prot_train_335_one  =windowing(prot_train_335_one,win_size)
label_train_335 =label(label_train_335)


prot_test_20  = windowing(prot_test_20,win_size)
prot_test_20_one  = windowing(prot_test_20_one,win_size)
label_test_20 = label(label_test_20)

prot_test_UB_31  = windowing(prot_test_UB_31,win_size)
prot_test_UB_31_one  = windowing(prot_test_UB_31_one,win_size)
label_test_UB_31 = label(label_test_UB_31)

prot_test_B_31  = windowing(prot_test_B_31,win_size)
prot_test_B_31_one  = windowing(prot_test_B_31_one,win_size)
label_test_B_31 = label(label_test_B_31)


In [16]:
prot_test_70=windowing(prot_test_70,win_size)
prot_test_70_one=windowing(prot_test_70_one,win_size)
label_test_70=label(label_test_70)


In [18]:
#######model training starts here

In [19]:


def shuffleData(X, y):
    index = [i for i in range(len(X))]
    random.shuffle(index)
    X = X[index]
    y = y[index]
    return X, y;
    
def shuffleData2(X1,X2, y):
    index = [i for i in range(len(X1))]
    random.shuffle(index)
    X1 = X1[index]
    X2 = X2[index]
    y = y[index]
    return X1, X2, y;

def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    
    return out

In [22]:


class model_class():
    def __init__(self,w_size):
        
        self.w_size=w_size
    

    def model_cnn_8(self):

        inputs = Input(shape = (self.w_size,1024,1))
        inputs2 = Input(shape = (self.w_size,20,1))


        c1 = Conv2D(32,(7,7), strides=(1,1), activation='relu', input_shape=(self.w_size, 1024,1),padding="same", kernel_regularizer=regularizers.l2(0.1))(inputs)


        c1 = LayerNormalization()(c1)
        c1 = Dropout(0.3,seed = 21)(c1)

        c1 = Conv2D(16,(5,5), strides=(1,1), activation='relu',padding="same", kernel_regularizer=regularizers.l2(0.05))(c1)

        c1 = LayerNormalization()(c1)

        c1 = Dropout(0.5,seed = 21)(c1)

        c1=Flatten()(c1)
        
        fc1 = Dense(1024, activation='relu')(c1)

        fc1 = Dense(512, activation='relu')(fc1)

        fc1 = Dense(256, activation='relu')(fc1)

        fc1 = Dense(128, activation='relu')(fc1)


        p1 = Flatten()(inputs2)
        
        fc2 = Dense(64, activation='relu')(p1)
        fc2 = Dense(32, activation='relu')(fc2)
        fc2 = Dense(16, activation='relu')(fc2)

 
        fc1 =concatenate([fc1,fc2])        


        fc1 = Dense(64, activation='relu')(fc1)

        fc1 = Dense(32, activation='relu')(fc1)

        fc1 = Dense(16, activation='relu')(fc1)

        fc1 = Dense(8, activation='relu')(fc1)

        fc2 = Dense(1, activation='sigmoid')(fc1)

        model1 = Model(inputs =[inputs,inputs2], outputs = [fc2])    
        opt=SGD(learning_rate=0.00001, momentum = 0.99)

        model1.compile(loss='binary_crossentropy', optimizer= opt, metrics=['accuracy'])

        return model1

In [23]:
def calculateScore(X, y, model): 
        
        i=0
        j=0
        for index in range(len(y)):
            if y[index]==0:
                i=i+1
            else :
                j=j+1
                
        score = model.evaluate(X,y)
        pred_y = model.predict(X)    
        accuracy = score[1];    
        tempLabel = np.zeros(shape = y.shape, dtype=np.int32)    
        for i in range(len(y)):
            if pred_y[i] < 0.5:
                tempLabel[i] = 0;
            else:
                tempLabel[i] = 1;
        confusion = confusion_matrix(y, tempLabel)
        TN, FP, FN, TP = confusion.ravel()
        
        sensitivity = recall_score(y, tempLabel)
        specificity = TN / float(TN+FP)
        
        # MCC = ((TP*TN)-(FP*FN))/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
        MCC=matthews_corrcoef(y, tempLabel)
        F1Score = (2 * TP) / float(2 * TP + FP + FN)
        precision = TP / float(TP + FP)    
        pred_y = pred_y.reshape((-1, ))    
        ROCArea = roc_auc_score(y, pred_y)
        recall = TP / float(TP + FN)
        fpr, tpr, thresholds = roc_curve(y, pred_y)
        lossValue = None;    
   
        y_true = tf.convert_to_tensor(y, np.float32)
        y_pred = tf.convert_to_tensor(pred_y, np.float32)
        PRAuc=average_precision_score(y, pred_y)   
        return {'sn' : sensitivity, 'sp' : specificity, 'acc' : accuracy, 'MCC' : MCC, 'AUC' : ROCArea,'PRAuc':PRAuc, 'precision' : precision, 'F1' : F1Score,  'lossValue' : lossValue}

In [29]:
model1=model_class(win_size)
model = model1.model_cnn_8() 

print(model.summary())

In [30]:


class_weights = compute_class_weight(class_weight = "balanced",classes = np.unique(label_train_335),y = label_train_335)
class_weights = dict(enumerate(class_weights))

history = model.fit([prot_train_335,prot_train_335_one], label_train_335, batch_size = 64,
                class_weight=class_weights,epochs=1)
      

In [31]:
# model = tf.keras.models.load_model('my_model_new.keras')
# model.save('my_model_315_tx_5.keras')

2024-03-20 07:40:43.707627: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78933 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:1d:00.0, compute capability: 8.0


In [33]:
print('test_60')
print(calculateScore([prot_test_60,prot_test_60_one], label_test_60, model))

test_60


2024-03-20 07:41:10.512767: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2024-03-20 07:41:11.259778: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-20 07:41:11.361173: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


411/411 [==============================] - 4s 8ms/step
{'sn': 0.42216867469879515, 'sp': 0.9209504020236697, 'acc': 0.8422093987464905, 'MCC': 0.3681679566204882, 'AUC': 0.7780495620570638, 'PRAuc': 0.4972832451096656, 'precision': 0.5002855511136494, 'F1': 0.45791949817041294, 'lossValue': None}


In [34]:
print('test_70')
print(calculateScore([prot_test_70,prot_test_70_one], label_test_70, model))

test_70
369/369 [==============================] - 3s 7ms/step
{'sn': 0.6921097770154374, 'sp': 0.9214504704514219, 'acc': 0.8760919570922852, 'MCC': 0.6111057533869702, 'AUC': 0.892935716789459, 'PRAuc': 0.7798232782501444, 'precision': 0.6847687738650827, 'F1': 0.6884197056941779, 'lossValue': None}


In [35]:
print('test_UB_31')
print(calculateScore([prot_test_UB_31,prot_test_UB_31_one], label_test_UB_31, model))


test_UB_31
208/208 [==============================] - 2s 7ms/step
{'sn': 0.5160523186682521, 'sp': 0.9249956992946843, 'acc': 0.873309314250946, 'MCC': 0.4347206452138929, 'AUC': 0.8331704758676737, 'PRAuc': 0.5457513056601294, 'precision': 0.49885057471264366, 'F1': 0.5073056691992986, 'lossValue': None}


In [36]:
print('test_B_31')
print(calculateScore([prot_test_B_31,prot_test_B_31_one], label_test_B_31, model))


test_B_31
206/206 [==============================] - 2s 8ms/step
{'sn': 0.5017261219792866, 'sp': 0.9264912280701755, 'acc': 0.8702998757362366, 'MCC': 0.43117979357261493, 'AUC': 0.8302801162861122, 'PRAuc': 0.5434115574273224, 'precision': 0.5099415204678363, 'F1': 0.505800464037123, 'lossValue': None}


In [37]:
print('test_20')
print(calculateScore([prot_test_20,prot_test_20_one], label_test_20, model)) 

test_20
142/142 [==============================] - 1s 8ms/step
{'sn': 0.20972354623450906, 'sp': 0.9298998569384835, 'acc': 0.7636443376541138, 'MCC': 0.194118186020322, 'AUC': 0.6379329861125317, 'PRAuc': 0.3525500827922383, 'precision': 0.4731182795698925, 'F1': 0.2906208718626156, 'lossValue': None}
